<b>Simple script to train Hidden Markov Model for Part of Speech tagging using NLTK</b>

In [ ]:
#import the models
import nltk
from nltk import HiddenMarkovModelTagger as hmm # do not use nltk.tag.hmm
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
import warnings
import dill
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns


warnings.filterwarnings('ignore')

<b>Download the data. Run only once</b>

In [ ]:
# Ensure the treebank dataset is downloaded
#nltk.download('treebank')
#nltk,download('punkt')
#nltk,download('brown')

<b>Prepare the data. We'll use the Penn Treebank which is an English Corpus that includes pos tagging. For information on the tagset: https://www.sketchengine.eu/penn-treebank-tagset/
We split the data into training and testing. Try to change the data size and experiment with the accuracy change.</b>


In [ ]:
print(f'The number of tagged examples in the dataset is: {len(brown.tagged_sents(tagset="universal"))}')
train_data = brown.tagged_sents(tagset='universal')[:50000]
test_data = brown.tagged_sents(tagset='universal')[50000:]

print(f'len of training data is {len(train_data)}')
print(f'len of testing data is {len(test_data)}')
print(train_data[0])

# Extracting unique tags from train_data
unique_tags = set(tag for sent in train_data for _, tag in sent)

print(unique_tags)

<b>Define the trainer and train the model</b>

In [ ]:
tagger = hmm.train(train_data, verbose=True)

In [ ]:
# Evaluate the model's accuracy on the test data
accuracy = tagger.accuracy(test_data)
print(f"Accuracy: {accuracy:.2f}")

<b>Generate true tags list and model prediction to get more detailed stats on where the model performed better and where it didn't perform so well</b>

In [ ]:
# Generate Predictions
true_tags = [tag for sent in test_data for _, tag in sent]
predicted_tags = [tag for sent in tagger.tag_sents([[word for word, _ in sent] for sent in test_data]) for _, tag in sent]
print(true_tags[20:40])
print(predicted_tags[20:40])

In [ ]:
# Accuracy
accuracy = accuracy_score(true_tags, predicted_tags)
print(f"Overall Accuracy: {accuracy:.2f}")

# Detailed classification report
print(classification_report(true_tags, predicted_tags))

# Confusion Matrix
conf_matrix = confusion_matrix(true_tags, predicted_tags, labels=labels)

In [ ]:
label_accuracy_dict = {}

for label in labels:
    correct_predictions = sum(1 for t, p in zip(true_tags, predicted_tags) if t == label and p == label)
    total_predictions = sum(1 for t in true_tags if t == label)
    wrong_predictions = total_predictions - correct_predictions
    label_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    
    # Store each label's accuracy in the dictionary
    label_accuracy_dict[label] = label_accuracy

    print(f"Label: {label}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Wrong Predictions: {wrong_predictions}")
    print(f"Accuracy: {label_accuracy:.2f}\n")

In [ ]:
# Bar plot of accuracy for each label
plt.figure(figsize=(12, 8))
plt.bar(label_accuracy_dict.keys(), label_accuracy_dict.values(), color='skyblue')
plt.xlabel('Tags')
plt.ylabel('Accuracy')
plt.title('Accuracy of Each Tag')
plt.xticks(rotation=45)
plt.show()

<b>If I'm happy with the model, I can save it for later usage</b>

In [ ]:
# Save the trained model to a file
with open('hmm_tagger.pkl', 'wb') as f:
    dill.dump(tagger, f)

<b>You can load the model at anytime to use it for tagging sentences</b>

In [ ]:
# Load the trained model from the file
with open('hmm_tagger.pkl', 'rb') as f:
    loaded_tagger = dill.load(f)

<b>Try the model on new text</b>

In [ ]:
sentence = 'I took the train from Zurich to Italy last night'

tokens = nltk.word_tokenize(sentence)

# Tag the tokenized sentence
tagged_sentence = loaded_tagger.tag(tokens)

print(tagged_sentence)